# Importando Bibliotecas

In [1]:
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig
import json

# Project Specification

In [ ]:
PROJECT_ID = "syntheticpersonasfinetuning"
REGION = "us-central1"
                        #project_number         #location            #Id_endpoint
ENDPOINT_ID = "projects/541997184461/locations/us-central1/endpoints/4205454954871128064" # (Confira se este é o ID do endpoint "Ligado")

# Prompt 

In [13]:
# 2. O PERFIL (O "GATILHO" DO SEU MODELO)
CLUSTER_3_PROFILE = """Is the most risk-averse and least open to novelty and challenge. They prefer stable and predictable environments, and they choose the safe option when confronted with probabilities. The strong value placed on job stability and security, along with a strong emphasis on work-life balance, reinforces their preference for safe options and an aversion to experimental environments. Financially, they demonstrate defensive and self-interested behavior in resource-sharing games."""

# 3. A PERGUNTA QUE VOCÊ QUER FAZER
# Prompt = """
# Act as an experienced screenwriter and social psychologist.
# Generate a deep, fictional persona that perfectly fits the system profile.
# The narrative should focus on how the person's life experiences shaped their attitudes.
# """


Prompt = """
Act as an experienced screenwriter and social psychologist.
Based *only* on the system profile, generate a fictional persona.

Output Requirements:
1.  Return ONLY a single, valid JSON object. No other text or markdown.
2.  The JSON object must have these exact keys: "name", "age" (integer 25-65), "department" (choose one from: [Technology, Sales, Customer Insights, Marketing, Business Intelligence & Strategy, Logistics, Finance, Human Resources (HR), Legal & Compliance, Trade Marketing, Investors Relation, Factory]), and "narrative_persona".
3.  The "narrative_persona" must be a complete, first-person biography of around 150 words that demonstrates the persona's attitudinal behaviors from their life experiences.

Respond with ONLY the JSON.
"""

# Model Config

In [15]:
# 4. INICIALIZAR E CHAMAR A API
vertexai.init(project=PROJECT_ID, location=REGION)
model = GenerativeModel(
    model_name=ENDPOINT_ID,
    system_instruction=CLUSTER_3_PROFILE
    )

In [ ]:
# ALLOWED_DEPARTMENTS = [
#     "Technology", "Sales", "Customer Insights", "Marketing",
#     "Business Intelligence & Strategy", "Logistics", "Finance",
#     "Human Resources (HR)", "Legal & Compliance", "Trade Marketing",
#     "Investors Relation", "Factory"
# ]


# # esquema do formato desejado de saída
# persona_schema = {
#     "type": "object",
#     "properties": {
#         "name": {
#             "type": "string",
#             "description": "A realistic full name for the persona."
#         },
#         "age": {
#             "type": "integer",
#             "description": "The persona's age as an integer, between 25 and 65.",
#             "minimum": 25,
#             "maximum": 65
#         },
#         "department": {
#             "type": "string",
#             "description": "The department where the persona works.",
#             "enum": ALLOWED_DEPARTMENTS  # <-- References the list above
#         },
#         "narrative_persona": {
#             "type": "string",
#             "description": "A first-person biography (approx. 150 words) explaining how life experiences led this person to have these opinions and attitudinal behaviors."
#         }
#     },
#     "required": ["name", "age", "department", "narrative_persona"]
# }


In [16]:
genaration_config = GenerationConfig(
    temperature=0.8, # temperatura que controla criatividade do modelo
    max_output_tokens=2048, # max de tokens palavras/partes de palavras
    # top_p = 0 a 1 soma a prob das palavras mais prováveis ate atingir um limite o top_p,
    top_k = 50, #limita a escolha para olhar apenas para as x palavras mais prováveis,
    # candidate_count = gera uma lista de x respostas diferentes para escolher,
    # stop_sequences = lista de que força o modelo a parara de gerar o texto se encontrar uma das strings dessa lista,
    response_mime_type="application/json", # Força o formato de saida do texto, (text/plain)
    #response_schema = persona_schema #fornece um esquema de como o json deve ser
    # presence_penalty =  penaliza palavras por ja terem aparecido na respostas incentivando a usar palavras novas 
    # seed pra reproduzir sempre a mesma resposta.
)

response = model.generate_content(
    Prompt,
    generation_config = genaration_config
)

In [17]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "{\n  \"name\": \"Eleanor Vance\",\n  \"age\": 42,\n  \"department\": \"Business Intelligence & Strategy\",\n  \"narrative_persona\": \"I\'ve always been the one who looks before leaping, and then looks again. My childhood was filled with carefully planned activities and a deep aversion to anything spontaneous. While other kids were chasing thrills, I was meticulously organizing my stamp collection or perfecting my homework. This cautious nature has only solidified with age. In my career in Business Intelligence & Strategy, I thrive on predictability. Give me a complex dataset, a meticulous process, and a clear goal, and I\'ll deliver. I\'m not one for brainstorming sessions or \\\"out-of-the-box\\\" thinking; I prefer a solid, data-driven plan. Job security is paramount, and I prioritize work-life balance above all else, ensuring my personal life remains as ordered and predictable as my professional one. I might not be

In [18]:
response.text

'{\n  "name": "Eleanor Vance",\n  "age": 42,\n  "department": "Business Intelligence & Strategy",\n  "narrative_persona": "I\'ve always been the one who looks before leaping, and then looks again. My childhood was filled with carefully planned activities and a deep aversion to anything spontaneous. While other kids were chasing thrills, I was meticulously organizing my stamp collection or perfecting my homework. This cautious nature has only solidified with age. In my career in Business Intelligence & Strategy, I thrive on predictability. Give me a complex dataset, a meticulous process, and a clear goal, and I\'ll deliver. I\'m not one for brainstorming sessions or \\"out-of-the-box\\" thinking; I prefer a solid, data-driven plan. Job security is paramount, and I prioritize work-life balance above all else, ensuring my personal life remains as ordered and predictable as my professional one. I might not be the life of the party, and I\'m definitely not the first to volunteer for a new, 

In [20]:
json_data = json.loads(response.text)

In [21]:
json_data

{'name': 'Eleanor Vance',
 'age': 42,
 'department': 'Business Intelligence & Strategy',
 'narrative_persona': 'I\'ve always been the one who looks before leaping, and then looks again. My childhood was filled with carefully planned activities and a deep aversion to anything spontaneous. While other kids were chasing thrills, I was meticulously organizing my stamp collection or perfecting my homework. This cautious nature has only solidified with age. In my career in Business Intelligence & Strategy, I thrive on predictability. Give me a complex dataset, a meticulous process, and a clear goal, and I\'ll deliver. I\'m not one for brainstorming sessions or "out-of-the-box" thinking; I prefer a solid, data-driven plan. Job security is paramount, and I prioritize work-life balance above all else, ensuring my personal life remains as ordered and predictable as my professional one. I might not be the life of the party, and I\'m definitely not the first to volunteer for a new, untested projec

In [ ]:
print(f"Name: {json_data['name']}")
print(f"Age: {json_data['age']}")
print(f"Department: {json_data['department']}")

In [ ]:
personas = [
    {   
        #'Cluster': 'Security_Seeker',
        'name': 'Eleanor Vance',
        'age': 42,
        'department': 'Business Intelligence & Strategy',
        'narrative_persona': 'I\'ve always been the one who looks before leaping, and then looks again. My childhood was filled with carefully planned activities and a deep aversion to anything spontaneous. While other kids were chasing thrills, I was meticulously organizing my stamp collection or perfecting my homework. This cautious nature has only solidified with age. In my career in Business Intelligence & Strategy, I thrive on predictability. Give me a complex dataset, a meticulous process, and a clear goal, and I\'ll deliver. I\'m not one for brainstorming sessions or "out-of-the-box" thinking; I prefer a solid, data-driven plan. Job security is paramount, and I prioritize work-life balance above all else, ensuring my personal life remains as ordered and predictable as my professional one. I might not be the life of the party, and I\'m definitely not the first to volunteer for a new, untested project, but you can always count on me for a well-reasoned, reliable solution.'
    }
]

In [4]:
with open("C:/Users/Administrador/OneDrive/OPER/2025/Gemini_persona/json/personas_gemini.json", "w", encoding="utf-8") as f:
    json.dump(personas, f, indent=4, ensure_ascii=False)

In [ ]:
with open("C:/Users/Administrador/OneDrive/OPER/2025/Gemini_persona/json/personas_gemini.json", 'r', encoding='utf-8') as f:

        dados = json.loads(f)

In [12]:
print(type(dados))

<class 'list'>


In [13]:
dados[0]['name']

'Eleanor Vance'

# Function Generate Personas

to use this you first need to authenticate via terminal you account after installing CLI

authenticate runing the following command:
gcloud auth application-default login

In [2]:
def personas_generation(endpoint_id, cluster_profile: str):
    """
    The endpoint_id should follow the aspect:
    projects/{project_number}/locations/{region}/endpoints/{endpoint_id}
    """    

    PROJECT_ID = "syntheticpersonasfinetuning"
    REGION = "us-central1"

    ENDPOINT_ID = f"projects/541997184461/locations/us-central1/endpoints/{endpoint_id}" 


    prompt = """
            Act as an experienced screenwriter and social psychologist.
            Based *only* on the system profile, generate a fictional persona.

            Output Requirements:
            1.  Return ONLY a single, valid JSON object. No other text or markdown.
            2.  The JSON object must have these exact keys: "name", "age" (choose an integer 25-65), "department" (choose one from: [Technology, Sales, Customer Insights, Marketing, Business Intelligence & Strategy, Logistics, Finance, Human Resources (HR), Legal & Compliance, Trade Marketing, Investors Relation, Factory]), and "narrative_persona".
            3.  The "narrative_persona" must be a complete, first-person biography of around 150 words that demonstrates the persona's attitudinal behaviors from their life experiences.

            Respond with ONLY the JSON.
            """
    

    # model config
    vertexai.init(project = PROJECT_ID, location=REGION)
    model = GenerativeModel(
        model_name = ENDPOINT_ID,
        system_instruction= cluster_profile
    )


    config_genarate = GenerationConfig(
        temperature=0.8,
        top_k = 60,
        response_mime_type= "application/json"
    )

    response = model.generate_content(
        prompt,
        generation_config = config_genarate
    )

    return response

In [3]:
# Cluster profiles and endpoint IDs

# Cluster 0
Pragmatic_Guardian = """Is composed of individuals who combine a concern for personal security and benevolence with a strong preference for stability and job security. 
                            They exhibit cooperative behavior (contributing substantially to the common good and being relatively generous when sharing money), but they also demand a reasonable return when others make proposals. 
                            There is a paradox: despite risk aversion in abstract beliefs, in concrete choices, they accept moderate risk—suggesting pragmatic and conditional behavior: they take risks when the reward is attractive."""
#endpoint_id_cluster1 = 

# Cluster 1
Ambitious_Innovator = """Is the most creative, curious people who enjoy challenges, but also possess a strong sense of sacrifice for the future (a willingness to invest in long-term goals) with strong preferences for a competitive salary and opportunities for rapid career growth. 
                          Socially, they show moderate altruism and prefer environments that support work-life balance and clear feedback. 
                          Financially, they contribute high percentages to collective projects (high cooperativism) and accept risk when a potential return exists."""
#endpoint_id_cluster1 =

# CLuster 2
Disciplined_Traditionalist = """Is conservative and disciplined: they value security, rules, meritocracy, and the stability of institutions. 
                          Paradoxically, in practical monetary contexts, they show a great willingness to cooperate and share resources (high contributions and donations in the scenarios), which indicates a high trust in the norms of reciprocity (they believe that cooperation pays off). 
                          They prioritize competitive salary and job stability and security, maintaining a conservative bias regarding novelty and risks."""
#endpoint_id_cluster2 = 


# CLuster 3
Security_Seeker = """Is the most risk-averse and least open to novelty and challenge. 
                          They prefer stable and predictable environments, and they choose the safe option when confronted with probabilities. 
                          The strong value placed on job stability and security, along with a strong emphasis on work-life balance, reinforces their preference for safe options and an aversion to experimental environments. 
                          Financially, they demonstrate defensive and self-interested behavior in resource-sharing games."""
endpoint_id_cluster3 = 6954726605520371712

In [4]:
generated_persona_c3 = personas_generation(endpoint_id_cluster3, Security_Seeker)
json.loads(generated_persona_c3.text)

c:\Users\Administrador\AppData\Local\Programs\Python\Python310\lib\site-packages\vertexai\generative_models\_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


{'name': 'Eleanor Vance',
 'age': 48,
 'department': 'Finance',
 'narrative_persona': "My life has always been a carefully constructed edifice of predictability. From a young age, I learned that the safest path is often the best one, a lesson reinforced by seeing others stumble when they strayed too far from the known. This worldview naturally guided my career choice towards a field where stability and job security are paramount. I’ve never been one to chase the latest trend or embrace untested ideas; I find comfort in established processes and clear expectations. Work-life balance isn't just a buzzword for me; it's a fundamental principle. I guard my personal time fiercely, recognizing that true security extends beyond the office. When resources are shared, I admit I tend to be a bit protective, ensuring my own needs are met before I consider broader distribution. It's not about being selfish, but about securing my foundation, which, for me, is the ultimate priority."}

# Salvando em Json todas as Personas

In [5]:
personas = [
    {   
        'Cluster': 'Security_Seeker',
        'name': 'Eleanor Vance',
        'age': 48,
        'department': 'Finance',
        'narrative_persona': "My life has always been a carefully constructed edifice of predictability. From a young age, I learned that the safest path is often the best one, a lesson reinforced by seeing others stumble when they strayed too far from the known. This worldview naturally guided my career choice towards a field where stability and job security are paramount. I’ve never been one to chase the latest trend or embrace untested ideas; I find comfort in established processes and clear expectations. Work-life balance isn't just a buzzword for me; it's a fundamental principle. I guard my personal time fiercely, recognizing that true security extends beyond the office. When resources are shared, I admit I tend to be a bit protective, ensuring my own needs are met before I consider broader distribution. It's not about being selfish, but about securing my foundation, which, for me, is the ultimate priority."
    }


#     {
#         #"Cluster" : "Ambtious Innovator"
#     },

#     {
#         #"Cluster" :
#     },

#     {
#         #"Cluster" :
#     }
]

In [6]:
with open("C:/Users/Administrador/OneDrive/OPER/2025/Gemini_persona/json/personas_gemini.json", "w", encoding="utf-8") as f:
    json.dump(personas, f, indent=4, ensure_ascii=False)